# Tugas pengelasan teks

Dalam modul ini, kita akan memulakan dengan tugas pengelasan teks yang mudah berdasarkan dataset **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: kita akan mengelaskan tajuk berita kepada salah satu daripada 4 kategori: Dunia, Sukan, Perniagaan dan Sains/Teknologi.

## Dataset

Untuk memuatkan dataset, kita akan menggunakan API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Kita kini boleh mengakses bahagian latihan dan ujian dataset dengan menggunakan `dataset['train']` dan `dataset['test']` masing-masing:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Mari cetak 10 tajuk berita baru pertama dari set data kita:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Pemvektoran Teks

Sekarang kita perlu menukar teks kepada **nombor** yang boleh diwakili sebagai tensor. Jika kita mahukan perwakilan pada peringkat perkataan, kita perlu melakukan dua perkara:

* Gunakan **tokenizer** untuk memecahkan teks kepada **token**.
* Bina **vokabulari** daripada token-token tersebut.

### Mengehadkan Saiz Vokabulari

Dalam contoh dataset AG News, saiz vokabulari agak besar, melebihi 100k perkataan. Secara amnya, kita tidak memerlukan perkataan yang jarang muncul dalam teks — hanya beberapa ayat sahaja yang akan mengandungi perkataan tersebut, dan model tidak akan belajar daripadanya. Oleh itu, adalah masuk akal untuk mengehadkan saiz vokabulari kepada jumlah yang lebih kecil dengan memberikan argumen kepada pembina vektorisasi:

Kedua-dua langkah ini boleh dikendalikan menggunakan lapisan **TextVectorization**. Mari kita wujudkan objek vektorisasi, dan kemudian panggil kaedah `adapt` untuk melalui semua teks dan membina vokabulari:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Nota** bahawa kami hanya menggunakan sebahagian kecil daripada keseluruhan set data untuk membina kosa kata. Kami melakukannya untuk mempercepatkan masa pelaksanaan dan supaya anda tidak perlu menunggu lama. Walau bagaimanapun, kami mengambil risiko bahawa beberapa perkataan daripada keseluruhan set data mungkin tidak dimasukkan ke dalam kosa kata, dan akan diabaikan semasa latihan. Oleh itu, menggunakan saiz kosa kata penuh dan menjalankan keseluruhan set data semasa `adapt` seharusnya meningkatkan ketepatan akhir, tetapi tidak secara signifikan.

Sekarang kita boleh mengakses kosa kata sebenar:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Dengan menggunakan penvektor, kita boleh dengan mudah mengekod sebarang teks ke dalam satu set nombor:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Representasi teks Bag-of-words

Oleh kerana perkataan mewakili makna, kadangkala kita boleh memahami maksud sesuatu teks hanya dengan melihat perkataan individu, tanpa menghiraukan susunan mereka dalam ayat. Sebagai contoh, apabila mengklasifikasikan berita, perkataan seperti *cuaca* dan *salji* mungkin menunjukkan *ramalan cuaca*, manakala perkataan seperti *saham* dan *dolar* akan merujuk kepada *berita kewangan*.

**Bag-of-words** (BoW) adalah representasi vektor tradisional yang paling mudah difahami. Setiap perkataan dihubungkan dengan indeks vektor, dan elemen vektor mengandungi bilangan kemunculan setiap perkataan dalam dokumen tertentu.

![Imej menunjukkan bagaimana representasi vektor bag-of-words diwakili dalam memori.](../../../../../translated_images/ms/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Anda juga boleh menganggap BoW sebagai jumlah semua vektor satu-hot-encoded untuk setiap perkataan dalam teks.

Di bawah adalah contoh bagaimana untuk menghasilkan representasi bag-of-words menggunakan pustaka python Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Kita juga boleh menggunakan penvektor Keras yang telah kita definisikan di atas, menukar setiap nombor perkataan kepada pengekodan satu-hot dan menambahkan semua vektor tersebut:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Nota**: Anda mungkin terkejut bahawa hasilnya berbeza daripada contoh sebelumnya. Sebabnya ialah dalam contoh Keras, panjang vektor sepadan dengan saiz perbendaharaan kata, yang dibina daripada keseluruhan dataset AG News, manakala dalam contoh Scikit Learn, kita membina perbendaharaan kata daripada teks sampel secara langsung.


## Melatih Pengklasifikasi BoW

Sekarang kita telah mempelajari cara membina representasi bag-of-words untuk teks kita, mari kita latih pengklasifikasi yang menggunakannya. Pertama, kita perlu menukar dataset kita kepada representasi bag-of-words. Ini boleh dilakukan dengan menggunakan fungsi `map` seperti berikut:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Sekarang mari kita tentukan rangkaian neural pengklasifikasi mudah yang mengandungi satu lapisan linear. Saiz input ialah `vocab_size`, dan saiz output sepadan dengan bilangan kelas (4). Oleh kerana kita sedang menyelesaikan tugas pengklasifikasian, fungsi pengaktifan terakhir ialah **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Memandangkan kita mempunyai 4 kelas, ketepatan melebihi 80% adalah hasil yang baik.

## Melatih pengklasifikasi sebagai satu rangkaian

Oleh kerana penvektor juga merupakan lapisan Keras, kita boleh mentakrifkan satu rangkaian yang merangkuminya, dan melatihnya secara end-to-end. Dengan cara ini, kita tidak perlu memvektor dataset menggunakan `map`, kita hanya boleh menghantar dataset asal ke input rangkaian.

> **Nota**: Kita masih perlu menggunakan `map` pada dataset kita untuk menukar medan daripada kamus (seperti `title`, `description` dan `label`) kepada tuple. Walau bagaimanapun, apabila memuatkan data daripada cakera, kita boleh membina dataset dengan struktur yang diperlukan sejak awal.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigram, trigram dan n-gram

Satu kekurangan pendekatan bag-of-words ialah beberapa perkataan adalah sebahagian daripada ungkapan berbilang perkataan. Sebagai contoh, perkataan 'hot dog' mempunyai makna yang berbeza sama sekali daripada perkataan 'hot' dan 'dog' dalam konteks lain. Jika kita sentiasa mewakili perkataan 'hot' dan 'dog' menggunakan vektor yang sama, ia boleh mengelirukan model kita.

Untuk menangani isu ini, **representasi n-gram** sering digunakan dalam kaedah pengelasan dokumen, di mana kekerapan setiap perkataan, dua perkataan atau tiga perkataan adalah ciri yang berguna untuk melatih pengelas. Dalam representasi bigram, sebagai contoh, kita akan menambah semua pasangan perkataan ke dalam perbendaharaan kata, selain daripada perkataan asal.

Di bawah adalah contoh bagaimana untuk menjana representasi bag-of-words bigram menggunakan Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Kelemahan utama pendekatan n-gram adalah saiz perbendaharaan kata mula berkembang dengan sangat pantas. Dalam praktiknya, kita perlu menggabungkan representasi n-gram dengan teknik pengurangan dimensi, seperti *embeddings*, yang akan kita bincangkan dalam unit seterusnya.

Untuk menggunakan representasi n-gram dalam dataset **AG News** kita, kita perlu memberikan parameter `ngrams` kepada pembina `TextVectorization` kita. Panjang perbendaharaan kata bigram adalah **jauh lebih besar**, dalam kes kita ia melebihi 1.3 juta token! Oleh itu, adalah masuk akal untuk menghadkan token bigram kepada jumlah yang munasabah.

Kita boleh menggunakan kod yang sama seperti di atas untuk melatih pengklasifikasi, namun, ia akan menjadi sangat tidak efisien dari segi memori. Dalam unit seterusnya, kita akan melatih pengklasifikasi bigram menggunakan embeddings. Sementara itu, anda boleh mencuba latihan pengklasifikasi bigram dalam notebook ini dan lihat jika anda boleh mendapatkan ketepatan yang lebih tinggi.


## Mengira Vektor BoW Secara Automatik

Dalam contoh di atas, kita mengira vektor BoW secara manual dengan menjumlahkan pengekodan satu-haba bagi setiap perkataan. Walau bagaimanapun, versi terkini TensorFlow membolehkan kita mengira vektor BoW secara automatik dengan memberikan parameter `output_mode='count` kepada pembina penvektor. Ini menjadikan proses mendefinisikan dan melatih model kita jauh lebih mudah:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Kekerapan istilah - kekerapan dokumen songsang (TF-IDF)

Dalam representasi BoW, kejadian perkataan diberi berat menggunakan teknik yang sama tanpa mengira perkataan itu sendiri. Walau bagaimanapun, jelas bahawa perkataan yang kerap muncul seperti *a* dan *in* adalah jauh kurang penting untuk pengelasan berbanding istilah khusus. Dalam kebanyakan tugas NLP, sesetengah perkataan lebih relevan daripada yang lain.

**TF-IDF** bermaksud **kekerapan istilah - kekerapan dokumen songsang**. Ia adalah variasi daripada bag-of-words, di mana nilai binari 0/1 yang menunjukkan kemunculan perkataan dalam dokumen digantikan dengan nilai titik terapung, yang berkaitan dengan kekerapan kemunculan perkataan dalam korpus.

Secara lebih formal, berat $w_{ij}$ bagi perkataan $i$ dalam dokumen $j$ ditakrifkan sebagai:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
di mana
* $tf_{ij}$ ialah bilangan kemunculan $i$ dalam $j$, iaitu nilai BoW yang telah kita lihat sebelum ini
* $N$ ialah bilangan dokumen dalam koleksi
* $df_i$ ialah bilangan dokumen yang mengandungi perkataan $i$ dalam keseluruhan koleksi

Nilai TF-IDF $w_{ij}$ meningkat secara berkadar dengan bilangan kali perkataan muncul dalam dokumen dan diselaraskan dengan bilangan dokumen dalam korpus yang mengandungi perkataan tersebut, yang membantu menyesuaikan fakta bahawa sesetengah perkataan muncul lebih kerap daripada yang lain. Sebagai contoh, jika perkataan muncul dalam *setiap* dokumen dalam koleksi, $df_i=N$, dan $w_{ij}=0$, dan istilah tersebut akan diabaikan sepenuhnya.

Anda boleh dengan mudah mencipta vektorisasi TF-IDF teks menggunakan Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Dalam Keras, lapisan `TextVectorization` boleh secara automatik mengira frekuensi TF-IDF dengan memberikan parameter `output_mode='tf-idf'`. Mari ulangi kod yang kita gunakan di atas untuk melihat sama ada penggunaan TF-IDF meningkatkan ketepatan:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Kesimpulan

Walaupun representasi TF-IDF memberikan berat kekerapan kepada pelbagai perkataan, ia tidak mampu mewakili makna atau susunan. Seperti yang dikatakan oleh ahli bahasa terkenal J. R. Firth pada tahun 1935, "Makna lengkap sesuatu perkataan sentiasa bersifat kontekstual, dan tiada kajian tentang makna yang terpisah daripada konteks boleh dianggap serius." Kita akan belajar bagaimana menangkap maklumat kontekstual daripada teks menggunakan pemodelan bahasa dalam kursus ini nanti.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
